In [1]:
import welding_machine as wm
from matplotlib import pyplot as plt
import random as rd
import FaDAm as fd
from numpy import mean
from numpy import std
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from keras.utils import to_categorical

In [2]:
x_path = "data/x_data.csv"

base_path = "data/"

current_file_name = '_current_data.csv'
voltage_file_name = '_voltage_data.csv'
wire_feed_file_name = '_wire_feed_data.csv'
y_file_name = "_y_data.csv"

n_time_steps = 128  # 0.1초 data * 100 = 10초
n_features = 3  # current, voltage, wire_feed

In [3]:
def load_LSTM_data(prefix=''):
    x_LSTM_data = list()
    x_LSTM_data.append(load_file(base_path+prefix+current_file_name))
    x_LSTM_data.append(load_file(base_path+prefix+voltage_file_name))
    x_LSTM_data.append(load_file(base_path+prefix+wire_feed_file_name))
    x_LSTM_data = np.dstack(x_LSTM_data)

    y_data = load_file(base_path+prefix+y_file_name)

    return x_LSTM_data, y_data

def load_file(file_path):
    data_frame = pd.read_csv(file_path, header=None, delim_whitespace=True)
    return data_frame.values

In [4]:
# fit and evaluate a model
def evaluate_model(train_x, train_y, test_x, test_y):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
    return accuracy


# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [5]:
train_x , train_y = load_LSTM_data('train')
test_x, test_y = load_LSTM_data('test')

In [6]:
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [7]:
verbose, epochs, batch_size = 0, 15, 64
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [8]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [10]:
_, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)

In [11]:
print(accuracy)

0.9035555720329285
